# Lab Assignment Seven: Recurrent Network Architectures
 

#### Everett Cienkus, Blake Miller, Colin Weil

## 1. Preparation

### 1.1 Define and Prepare Class Variables

### 1.2 Choose Metric for Evaluating Peformance

### 1.3 Choose Method for Dividing Data

## 2. Modeling

### 2.1 Investigate Two Different Network Architectures

#### 2.1.1 LSTM

#### 2.1.2 GRU

#### 2.1.3 Discussion and Comparison of RNN's

### 2.2
(FREE POINTS WOOHOO +1)

### 2.3 Visualize Results of RNN's

## 3. Exceptional Work 
(FREE POINTS WOOHOO +1)